# Image warping

* Learn to apply different geometric transformations to images, like translation, rotation, affine transformation etc.

OpenCV provides two transformation functions, `cv.warpAffine` and `cv.warpPerspective`, with which you can perform all kinds of transformations. `cv.warpAffine` takes a 2x3 transformation matrix while `cv.warpPerspective` takes a 3x3 transformation matrix as input.

## OpenCV

(pytorch)$ conda install -c conda-forge py-opencv

In [ ]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt

plt.rcParams['figure.figsize'] = [16, 9]

## Scaling

Scaling is just resizing of the image.
OpenCV comes with a function `cv.resize()` for this purpose.
The size of the image can be specified manually, or you can specify the scaling factor.
Different interpolation methods are used.
Preferable interpolation methods are `cv.INTER_AREA` for shrinking and `cv.INTER_CUBIC` (slow) & `cv.INTER_LINEAR` for zooming.
By default, the interpolation method `cv.INTER_LINEAR` is used for all resizing purposes.

* Resizes an image.
    * `dst = cv.resize(src, dsize[, dst[, fx[, fy[, interpolation]]]])`
        * dsize: output image size
        * fx: scale factor along the horizontal axis
        * fy: scale factor along the vertical axis;

In [ ]:
img = cv.imread('messi5.jpg')

height, width = img.shape[:2]
# scale image (1.5x width, 2x height)
# TODO
dst = ...

plt.subplot(121),plt.imshow(cv.cvtColor(img,cv.COLOR_BGR2RGB)),plt.title('Input')
plt.subplot(122),plt.imshow(cv.cvtColor(dst,cv.COLOR_BGR2RGB)),plt.title('Output')
plt.show()

## Translation

Translation is the shifting of an object's location.
If you know the shift in the (x,y) direction and let it be (tx,ty), you can create the transformation matrix M as follows:

$M = \begin{bmatrix}1 & 0 & t_x\\ 0 & 1 & t_y\end{bmatrix}$

You can take make it into a Numpy array of type np.float32 and pass it into the cv.warpAffine() function.

* Applies an affine transformation to an image.
    * `dst = cv.warpAffine( src, M, dsize[, dst[, flags[, borderMode[, borderValue]]]])`
    * warning: The third argument of the cv.warpAffine() function is the size of the output image, which should be in the form of **(width, height)**. Remember width = number of columns, and height = number of rows.

In [ ]:
img = cv.imread('messi5.jpg',0)

# translate image (translation of (100,50))
# TODO
rows,cols = img.shape
M = ...
dst = cv.warpAffine(img,M,(cols,rows))


plt.subplot(121),plt.imshow(cv.cvtColor(img,cv.COLOR_BGR2RGB)),plt.title('Input')
plt.subplot(122),plt.imshow(cv.cvtColor(dst,cv.COLOR_BGR2RGB)),plt.title('Output')
plt.show()

## Rotation

Rotation of an image for an angle θ is achieved by the transformation matrix of the form

$M = \begin{bmatrix} cos\theta & -sin\theta \\ sin\theta & cos\theta \end{bmatrix}$

But OpenCV provides scaled rotation with adjustable center of rotation so that you can rotate at any location you prefer. The modified transformation matrix is given by

$M = \begin{bmatrix} \alpha & \beta & (1- \alpha ) \cdot center.x - \beta \cdot center.y \\ - \beta & \alpha & \beta \cdot center.x + (1- \alpha ) \cdot center.y\end{bmatrix}$

where:

$ \alpha = scale \cdot \cos \theta , \\ \beta = scale \cdot \sin \theta$

To find this transformation matrix, OpenCV provides a function, `cv.getRotationMatrix2D`. Check out the below example which rotates the image by 90 degree with respect to center without any scaling.

* Calculates an affine matrix of 2D rotation.
    * `retval = cv.getRotationMatrix2D(center, angle, scale)`

* Applies an affine transformation to an image.
    * `dst = cv.warpAffine( src, M, dsize[, dst[, flags[, borderMode[, borderValue]]]])`
    * warning: The third argument of the cv.warpAffine() function is the size of the output image, which should be in the form of **(width, height)**. Remember width = number of columns, and height = number of rows.

In [ ]:
img = cv.imread('messi5.jpg',0)

# compute rotation matrix and rotate image (center should be image center, angle=45, scale=1)
# TODO
rows,cols = img.shape # cols-1 and rows-1 are the coordinate limits.
M = ...
dst = ...


plt.subplot(121),plt.imshow(cv.cvtColor(img,cv.COLOR_BGR2RGB)),plt.title('Input')
plt.subplot(122),plt.imshow(cv.cvtColor(dst,cv.COLOR_BGR2RGB)),plt.title('Output')
plt.show()

## Affine Transformation

In affine transformation, all parallel lines in the original image will still be parallel in the output image. To find the transformation matrix, we need three points from the input image and their corresponding locations in the output image. Then cv.getAffineTransform will create a 2x3 matrix which is to be passed to cv.warpAffine.

Check the below example, and also look at the green points

* Calculates an affine transform from three pairs of the corresponding points.
    * `retval = cv.getAffineTransform(src, dst)`

* Applies an affine transformation to an image.
    * `dst = cv.warpAffine( src, M, dsize[, dst[, flags[, borderMode[, borderValue]]]])`
    * warning: The third argument of the cv.warpAffine() function is the size of the output image, which should be in the form of **(width, height)**. Remember width = number of columns, and height = number of rows.

In [ ]:
img = cv.imread('drawing.png')

pts1 = np.float32([[50,50],[200,50],[50,200]])
pts2 = np.float32([[10,100],[200,50],[100,250]])

# draw points
for p in pts1:
    p = p.astype('int32')
    img = cv.circle(img, tuple(p), radius=0, color=(0, 255, 0), thickness=8)

# compute affine transformation matrix and transform image
# TODO
rows,cols,ch = img.shape
M = ...
dst = ...


plt.subplot(121),plt.imshow(cv.cvtColor(img,cv.COLOR_BGR2RGB)),plt.title('Input')
plt.subplot(122),plt.imshow(cv.cvtColor(dst,cv.COLOR_BGR2RGB)),plt.title('Output')
plt.show()

## Perspective Transformation

For perspective transformation, you need a 3x3 transformation matrix.
Straight lines will remain straight even after the transformation.
To find this transformation matrix, you need 4 points on the input image and corresponding points on the output image.
Among these 4 points, 3 of them should not be collinear.
Then the transformation matrix can be found by the function cv.getPerspectiveTransform.
Then apply cv.warpPerspective with this 3x3 transformation matrix.

* Calculates a perspective transform from four pairs of the corresponding points.
    * `retval = cv.getPerspectiveTransform(src, dst[, solveMethod])`

* Applies a perspective transformation to an image:
    * `dst = cv.warpPerspective(src, M, dsize[, dst[, flags[, borderMode[, borderValue]]]])`

In [ ]:
img = cv.imread('sudoku.png')


pts1 = np.float32([[72,87],[490,70],[40,513],[519,519]])
pts2 = np.float32([[0,0],[300,0],[0,300],[300,300]])
# draw points
for p in pts1:
    p = p.astype('int32')
    img = cv.circle(img, tuple(p), radius=0, color=(0, 255, 0), thickness=8)

rows,cols,ch = img.shape
# compute perspective transformation matrix and transform image (dsize=(300,300))
# TODO
M = ...
dst = cv.warpPerspective(img,M,(300,300))

plt.subplot(121),plt.imshow(cv.cvtColor(img,cv.COLOR_BGR2RGB)),plt.title('Input')
plt.subplot(122),plt.imshow(cv.cvtColor(dst,cv.COLOR_BGR2RGB)),plt.title('Output')
plt.show()

## Scaling

Use cv.warpPerspective() function.

In [ ]:
img = cv.imread('messi5.jpg',0)

# scale image (0.7x width, 0.5x height)
# TODO
print(img.shape)
rows,cols = img.shape
M_scale = ...
dst = cv.warpPerspective(img,M_scale,(cols,rows))


plt.subplot(121),plt.imshow(cv.cvtColor(img,cv.COLOR_BGR2RGB)),plt.title('Input')
plt.subplot(122),plt.imshow(cv.cvtColor(dst,cv.COLOR_BGR2RGB)),plt.title('Output')
plt.show()

## Shear

Use cv.warpPerspective() function.

In [ ]:
img = cv.imread('messi5.jpg',0)

# shear image x:0.3
# TODO
print(img.shape)
rows,cols = img.shape
M_shear = ...
dst = ...


plt.subplot(121),plt.imshow(cv.cvtColor(img,cv.COLOR_BGR2RGB)),plt.title('Input')
plt.subplot(122),plt.imshow(cv.cvtColor(dst,cv.COLOR_BGR2RGB)),plt.title('Output')
plt.show()

## Translation

Use cv.warpPerspective() function.

In [ ]:
img = cv.imread('messi5.jpg',0)

# translate image (translation of (100,50))
# TODO
rows,cols = img.shape
M_translate = ...
dst = ...


plt.subplot(121),plt.imshow(cv.cvtColor(img,cv.COLOR_BGR2RGB)),plt.title('Input')
plt.subplot(122),plt.imshow(cv.cvtColor(dst,cv.COLOR_BGR2RGB)),plt.title('Output')
plt.show()

## Rotation

- Use cv.warpPerspective() function.
- Don't use cv.getRotationMatrix2D() function
- `np.radians(degree)`

In [ ]:
img = cv.imread('messi5.jpg',0)

# compute rotation matrix and rotate image (center should be image center, angle=45, scale=1)
# TODO
rows,cols = img.shape # cols-1 and rows-1 are the coordinate limits.
degree = 15
M_rotate = ...
dst = ...

plt.subplot(121),plt.imshow(cv.cvtColor(img,cv.COLOR_BGR2RGB)),plt.title('Input')
plt.subplot(122),plt.imshow(cv.cvtColor(dst,cv.COLOR_BGR2RGB)),plt.title('Output')
plt.show()

## Composition

- Use cv.warpPerspective() function.
- Apply M_scale, M_shear, M_translate, M_rotate in that order

In [ ]:
img = cv.imread('messi5.jpg',0)

# compute rotation matrix and rotate image (center should be image center, angle=45, scale=1)
# TODO
rows,cols = img.shape # cols-1 and rows-1 are the coordinate limits.
M = ...
dst = ...

plt.subplot(121),plt.imshow(cv.cvtColor(img,cv.COLOR_BGR2RGB)),plt.title('Input')
plt.subplot(122),plt.imshow(cv.cvtColor(dst,cv.COLOR_BGR2RGB)),plt.title('Output')
plt.show()

## Affine Matrix

- Implement getAffineTransform() function
- Hints
    - Inverse Matrix : np.linalg.inv(mat) 
    - Matrix Multiplication : np.matmul(mat1, mat2)

In [ ]:
img = cv.imread('drawing.png')

pts1 = np.float32([[50,50],[200,50],[50,200]])
pts2 = np.float32([[10,100],[200,50],[100,250]])

# draw points
for p in pts1:
    p = p.astype('int32')
    img = cv.circle(img, tuple(p), radius=0, color=(0, 255, 0), thickness=8)

rows,cols,ch = img.shape

def getAffineTransform(src_pts, target_pts):
    # TODO
    ...

M = cv.getAffineTransform(pts1,pts2)
dst1 = cv.warpAffine(img,M,(cols,rows))

M = getAffineTransform(pts1,pts2)
dst2 = cv.warpPerspective(img,M,(cols,rows))

plt.subplot(131),plt.imshow(cv.cvtColor(img,cv.COLOR_BGR2RGB)),plt.title('Input')
plt.subplot(132),plt.imshow(cv.cvtColor(dst1,cv.COLOR_BGR2RGB)),plt.title('OpenCV results')
plt.subplot(133),plt.imshow(cv.cvtColor(dst2,cv.COLOR_BGR2RGB)),plt.title('our results')
plt.show()

## Projective Matrix

- Implement getPerspectiveTransform() function
- Hints
    - Inverse Matrix : np.linalg.inv(mat) 
    - Matrix Multiplication : np.matmul(mat1, mat2)

In [ ]:
img = cv.imread('sudoku.png')


pts1 = np.float32([[72,87],[490,70],[40,513],[519,519]])
pts2 = np.float32([[0,0],[300,0],[0,300],[300,300]])

# draw points
for p in pts1:
    p = p.astype('int32')
    img = cv.circle(img, tuple(p), radius=0, color=(0, 255, 0), thickness=8)

## Affine Matrix
rows,cols,ch = img.shape


# compute perspective transformation matrix and transform image (dsize=(300,300))
# TODO
def getPerspectiveTransform(src_pts, target_pts):
    # TODO
    ...

M = cv.getPerspectiveTransform(pts1,pts2)
dst1 = cv.warpPerspective(img,M,(300,300))

M = getPerspectiveTransform(pts1,pts2)
dst2 = cv.warpPerspective(img,M,(300,300))

plt.subplot(131),plt.imshow(cv.cvtColor(img,cv.COLOR_BGR2RGB)),plt.title('Input')
plt.subplot(132),plt.imshow(cv.cvtColor(dst1,cv.COLOR_BGR2RGB)),plt.title('OpenCV results')
plt.subplot(133),plt.imshow(cv.cvtColor(dst2,cv.COLOR_BGR2RGB)),plt.title('our results')
plt.show()